In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

In [47]:
df1 = pd.read_json('project_data/schoolInfo.json')

In [48]:
df1.head()

,rankingNoteText,nonResponderText,nonResponder,act-avg,primaryPhoto,primaryPhotoThumb,sat-avg,enrollment,city,sortName,...,displayName,schoolType,region,aliasNames,rankingType,overallRank,institutionalControl,rankingMaxPossibleScore,rankingRankStatus,primaryKey
0,NaN,None,False,32.0,https://www.usnews.com/img/college-photo_31291...,https://www.usnews.com/img/college-photo_31291...,1400.0,5400.0,Princeton,princetonuniversity,...,Princeton University,national-universities,NaN,None,national-universities,1,private,NaN,ranked,2627
1,NaN,None,False,32.0,https://www.usnews.com/img/college-photo_8866.jpg,https://www.usnews.com/img/college-photo_8866_...,1430.0,6710.0,Cambridge,harvarduniversity,...,Harvard University,national-universities,NaN,None,national-universities,2,private,NaN,ranked,2155
2,NaN,None,False,32.0,https://www.usnews.com/dims4/USNEWS/5b128f0/17...,https://www.usnews.com/dims4/USNEWS/196469f/17...,1450.0,5941.0,Chicago,universityofchicago,...,University of Chicago,national-universities,NaN,UChicago,national-universities,3,private,NaN,ranked,1774
3,NaN,None,False,32.0,https://www.usnews.com/dims4/USNEWS/60348dd/17...,https://www.usnews.com/dims4/USNEWS/1ca230f/17...,1420.0,5472.0,New Haven,yaleuniversity,...,Yale University,national-universities,NaN,None,national-universities,3,private,NaN,ranked,1426
4,NaN,None,False,32.0,https://www.usnews.com/img/college-photo_19002...,https://www.usnews.com/img/college-photo_19002...,1430.0,6113.0,New York,columbiauniversity,...,Columbia University,national-universities,NaN,None,national-universities,5,private,NaN,ranked,2707


In [49]:
df1.isna().sum()

rankingNoteText            311
nonResponderText           300
nonResponder                 0
act-avg                     20
primaryPhoto                47
primaryPhotoThumb           47
sat-avg                     20
enrollment                  11
city                         0
sortName                     0
zip                          0
rankingNoteCharacter       311
acceptance-rate              9
rankingDisplayScore         81
percent-receiving-aid      168
cost-after-aid             168
state                        0
rankingSortRank              0
hs-gpa-avg                  67
urlName                      0
rankingDisplayName           0
rankingDisplayRank           0
ranking                      0
xwalkId                      0
rankingIsTied                0
isPublic                     0
businessRepScore            77
tuition                      0
engineeringRepScore        105
displayName                  0
schoolType                   0
region                     311
aliasNam

In [50]:
# Drop columns which only have NaN values
df1.dropna(axis=1, how='all', inplace=True)

In [51]:
df1.shape

(311, 35)

In [52]:
df1.isna().sum()

nonResponderText         300
nonResponder               0
act-avg                   20
primaryPhoto              47
primaryPhotoThumb         47
sat-avg                   20
enrollment                11
city                       0
sortName                   0
zip                        0
acceptance-rate            9
rankingDisplayScore       81
percent-receiving-aid    168
cost-after-aid           168
state                      0
rankingSortRank            0
hs-gpa-avg                67
urlName                    0
rankingDisplayName         0
rankingDisplayRank         0
ranking                    0
xwalkId                    0
rankingIsTied              0
isPublic                   0
businessRepScore          77
tuition                    0
engineeringRepScore      105
displayName                0
schoolType                 0
aliasNames               167
rankingType                0
overallRank                0
institutionalControl       0
rankingRankStatus          0
primaryKey    

In [104]:
# Drop irrelevant variables
df1 = df1.drop(['nonResponderText', 'nonResponder', 'primaryPhoto', 'primaryPhotoThumb', 'aliasNames', 'urlName'], 
         axis = 1)
df1.shape

(311, 29)

In [107]:
# These columns only have 1 unique value
unique_counts = df1.nunique()
single_value_cols = unique_counts[unique_counts == 1].index.tolist()
single_value_cols

['rankingDisplayName', 'ranking', 'isPublic', 'schoolType', 'rankingType']

In [55]:
df2 = pd.read_csv('project_data/MERGED2017_18_PP.csv')

/var/folders/vk/7c21x7qj4fv7flh8p9m2wwn00000gn/T/ipykernel_99604/539635706.py:1: DtypeWarning: Columns (1729,1909,1910,1911,1912,1913) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('project_data/MERGED2017_18_PP.csv')


In [56]:
df2.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,COUNT_WNE_MALE1_P8,MD_EARN_WNE_MALE1_P8,GT_THRESHOLD_P10,MD_EARN_WNE_INC1_P10,MD_EARN_WNE_INC2_P10,MD_EARN_WNE_INC3_P10,MD_EARN_WNE_INDEP1_P10,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df2.shape

(7112, 2989)

In [58]:
df2.isna().sum()

UNITID                       0
OPEID                        0
OPEID6                       0
INSTNM                       0
CITY                         0
                          ... 
MD_EARN_WNE_INC3_P10      7112
MD_EARN_WNE_INDEP1_P10    7112
MD_EARN_WNE_INDEP0_P10    7112
MD_EARN_WNE_MALE0_P10     7112
MD_EARN_WNE_MALE1_P10     7112
Length: 2989, dtype: int64

In [59]:
df2.dropna(axis=1, how='all', inplace=True)

In [60]:
df2.shape

(7112, 1689)

In [61]:
df2.isna().sum()

UNITID                    0
OPEID                     0
OPEID6                    0
INSTNM                    0
CITY                      0
                       ... 
OMACHT8_PELL_PTNFT     3139
OMAWDP8_PELL_PTNFT     4222
OMENRYP8_PELL_PTNFT    4222
OMENRAP8_PELL_PTNFT    4222
OMENRUP8_PELL_PTNFT    4222
Length: 1689, dtype: int64

In [73]:
df1.primaryKey

0        2627
1        2155
2        1774
3        1426
4        2707
        ...  
306     41279
307     10923
308    209885
309     25042
310      7948
Name: primaryKey, Length: 311, dtype: int64

In [81]:
df2['primaryKey'] = df2['OPEID6']

In [87]:
result = pd.merge(df1, df2, on='primaryKey', how="inner", indicator = True)
result.shape

(395, 1726)

In [113]:
result.head()

,nonResponderText,nonResponder,act-avg,primaryPhoto,primaryPhotoThumb,sat-avg,enrollment,city,sortName,zip,...,OMENRYP8_PELL_FTNFT,OMENRAP8_PELL_FTNFT,OMENRUP8_PELL_FTNFT,OMACHT8_PELL_PTNFT,OMAWDP8_PELL_PTNFT,OMENRYP8_PELL_PTNFT,OMENRAP8_PELL_PTNFT,OMENRUP8_PELL_PTNFT,displayName_y,_merge
0,None,False,32.0,https://www.usnews.com/img/college-photo_31291...,https://www.usnews.com/img/college-photo_31291...,1400.0,5400.0,Princeton,princetonuniversity,8544,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Princeton University,both
1,None,False,32.0,https://www.usnews.com/img/college-photo_8866.jpg,https://www.usnews.com/img/college-photo_8866_...,1430.0,6710.0,Cambridge,harvarduniversity,2138,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Harvard University,both
2,None,False,32.0,https://www.usnews.com/dims4/USNEWS/5b128f0/17...,https://www.usnews.com/dims4/USNEWS/196469f/17...,1450.0,5941.0,Chicago,universityofchicago,60637,...,0.0000,0.0,0.1333,0.0,NaN,NaN,NaN,NaN,University of Chicago,both
3,None,False,32.0,https://www.usnews.com/dims4/USNEWS/60348dd/17...,https://www.usnews.com/dims4/USNEWS/1ca230f/17...,1420.0,5472.0,New Haven,yaleuniversity,6520,...,0.0000,0.0,0.0000,0.0,NaN,NaN,NaN,NaN,Yale University,both
4,None,False,32.0,https://www.usnews.com/img/college-photo_19002...,https://www.usnews.com/img/college-photo_19002...,1430.0,6113.0,New York,columbiauniversity,10027,...,0.0103,0.0,0.1598,70.0,0.7857,0.0,0.0143,0.2,Columbia University in the City of New York,both


In [114]:
duplicates = df2[df2.duplicated(['primaryKey'])]
duplicates.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,SCH_DEG,MAIN,NUMBRANCH,...,OMENRYP8_PELL_FTNFT,OMENRAP8_PELL_FTNFT,OMENRUP8_PELL_FTNFT,OMACHT8_PELL_PTNFT,OMAWDP8_PELL_PTNFT,OMENRYP8_PELL_PTNFT,OMENRAP8_PELL_PTNFT,OMENRUP8_PELL_PTNFT,displayName,primaryKey
28,101499,106010,1060,Jefferson Davis Community College,Brewton,AL,36426,2.0,0,3,...,0.0510,0.1699,0.3992,232.0,0.3793,0.0690,0.1681,0.3836,Jefferson Davis Community College,1060
43,101949,106006,1060,Alabama Southern Community College,Monroeville,AL,36460,2.0,0,3,...,0.0408,0.5714,0.0408,26.0,0.3846,0.0385,0.5769,0.0000,Alabama Southern Community College,1060
71,103909,2100611,21006,Carrington College-Mesa,Mesa,AZ,85210-5005,1.0,0,6,...,0.0000,0.1079,0.1162,0.0,NaN,NaN,NaN,NaN,Carrington College-Mesa,21006
72,103927,2100607,21006,Carrington College-Tucson,Tucson,AZ,85745-2974,1.0,0,6,...,0.0000,0.1023,0.1932,0.0,NaN,NaN,NaN,NaN,Carrington College-Tucson,21006
77,104151,108100,1081,Arizona State University Campus Immersion,Tempe,AZ,85287,3.0,1,10,...,0.0072,0.1686,0.1261,334.0,0.5359,0.0150,0.2725,0.1766,Arizona State University Campus Immersion,1081


In [115]:
duplicates_in_df1 = duplicates[duplicates['primaryKey'].isin(df1['primaryKey'])]
duplicates_in_df1.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,SCH_DEG,MAIN,NUMBRANCH,...,OMENRYP8_PELL_FTNFT,OMENRAP8_PELL_FTNFT,OMENRUP8_PELL_FTNFT,OMACHT8_PELL_PTNFT,OMAWDP8_PELL_PTNFT,OMENRYP8_PELL_PTNFT,OMENRAP8_PELL_PTNFT,OMENRUP8_PELL_PTNFT,displayName,primaryKey
77,104151,108100,1081,Arizona State University Campus Immersion,Tempe,AZ,85287,3.0,1,10,...,0.0072,0.1686,0.1261,334.0,0.5359,0.015,0.2725,0.1766,Arizona State University Campus Immersion,1081
365,117928,2179943,21799,Argosy University-The Art Institute of Califor...,San Francisco,CA,94102-4908,3.0,0,26,...,0.0000,0.1059,0.5412,107.0,0.2430,0.000,0.0187,0.7383,Argosy University-The Art Institute of Califor...,21799
439,121983,2179908,21799,Argosy University-San Francisco Bay Area,Alameda,CA,94501,3.0,0,26,...,0.0000,0.0000,0.2000,15.0,0.3333,0.000,0.0000,0.6667,Argosy University-San Francisco Bay Area,21799
518,126030,2179937,21799,Western State College of Law at Argosy University,Irvine,CA,92618-3601,3.0,0,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western State College of Law at Argosy University,21799
727,135610,177201,1772,Trinity International University-Florida,Miami,FL,33186,3.0,0,3,...,0.0000,0.1429,0.3571,40.0,0.5000,0.025,0.0750,0.4000,Trinity International University-Florida,1772
